# Data Cleaning and Data Preparation Part

In [73]:
import numpy as np
import pandas as pd
import os

path = os.getcwd()

### Load all the datasets related to CKD challenge test

In [74]:
creatinine = pd.read_csv("T_creatinine.csv")
DBP = pd.read_csv("T_DBP.csv")
demo = pd.read_csv("T_demo.csv")
glucose = pd.read_csv("T_glucose.csv")
HGB = pd.read_csv("T_HGB.csv")
ldl = pd.read_csv("T_ldl.csv")
meds = pd.read_csv("T_meds.csv")
SBP = pd.read_csv("T_SBP.csv")
stage = pd.read_csv("T_stage.csv")

 
 ## Lets create a generic method for data preparation

In [75]:
def prep_lab_test_df(df, lab_test):
    new_df = df.groupby('id', as_index=False).agg(
        no_of_test = ("value", "count"), 
        mean_value = ("value", "mean"), 
        duration_of_test = ("time", sum))

    new_df.insert(loc=0, column='id', value=new_df.index)
    print("Summary of " + lab_test + " lab test below:")
    print(new_df.describe())
    print("\n \nShape of " + lab_test + " lab test below:")
    print(new_df.shape)
    print("\n \nFirst four records of " + lab_test + " lab test below:")
    print(new_df.head())
    return new_df

### Below is the days count for a particular medicine with their dosage

In [76]:
meds = pd.read_csv("T_meds.csv")

meds['no_of_days_with_drug'] = 0
meds['no_of_days_without_drug'] = 0

In [77]:
flag = True
temp = " "
for i in range(len(meds)):
    if meds.loc[i,'end_day'] < 0:
        meds.loc[i,'no_of_days_with_drug'] = 0
    else:
        meds.loc[i,'no_of_days_with_drug'] = meds.loc[i,'end_day'] - meds.loc[i,'start_day']
        
    if meds.loc[i,'drug'] != temp and flag:
        if meds.loc[i,'start_day'] < 0:
            meds.loc[i,'no_of_days_without_drug'] = 0
        else:
            meds.loc[i,'no_of_days_without_drug'] = meds.loc[i,'start_day']
        temp = meds.loc[i,'drug']
    elif meds.loc[i,'drug'] == temp:
        meds.loc[i,'no_of_days_without_drug'] = meds.loc[i,'start_day'] - meds.loc[i-1,'end_day']
        if meds.loc[i,'no_of_days_without_drug'] < 0:
            #meds.loc[i,'no_of_days_with_drug'] = meds.loc[i,'no_of_days_with_drug'] + meds.loc[i,'no_of_days_without_drug']
            meds.loc[i,'no_of_days_without_drug'] = 0
        
    if i+1 < len(meds) and meds.loc[i+1,'drug'] == temp:
        flag = False
    else:
        flag = True

In [78]:
meds.to_csv("meds.csv", index=False)

In [79]:
meds1 = meds.groupby('id').agg(
        total_no_of_days_with_drug = ("no_of_days_with_drug", sum), 
        total_no_of_days_without_drug = ("no_of_days_without_drug", sum))
meds1 = meds1.reset_index()
meds1.shape

(272, 3)

In [80]:
meds1.head()

,id,total_no_of_days_with_drug,total_no_of_days_without_drug
0,0,1290,328
1,1,300,221
2,2,180,487
3,3,360,301
4,4,1620,481


### Number of missing medication for patient

In [81]:
id1 = meds.id.unique()
id2 = demo.id.unique()
meds_details_not_available = set(id2) - set(id1)
print(len(meds_details_not_available))

28


In [82]:
meds.head()

,id,drug,daily_dosage,start_day,end_day,no_of_days_with_drug,no_of_days_without_drug
0,0,atorvastatin,10.0,19,109,90,19
1,0,atorvastatin,10.0,117,207,90,8
2,0,losartan,100.0,19,289,270,19
3,0,losartan,100.0,403,493,90,114
4,0,losartan,100.0,587,677,90,94


### Total no of days with drug for a patient

In [83]:
new_df = meds.groupby(['id', 'drug']).agg(
        total_no_of_days_with_drug = ("no_of_days_with_drug", sum))
new_df = new_df.reset_index()

In [84]:
new_df.shape

(611, 3)

In [85]:
new_df.head()

,id,drug,total_no_of_days_with_drug
0,0,atorvastatin,180
1,0,losartan,450
2,0,metformin,660
3,1,atorvastatin,300
4,2,rosuvastatin,180


### Converting long to wide format (for each patient, and their corresponding medicine

In [86]:
df_reshape = new_df.pivot(index='id', columns='drug', values='total_no_of_days_with_drug')
df_reshape.insert(loc=0, column='id', value=df_reshape.index)
df_reshape.index = df_reshape.index.set_names(['index'])
df_reshape.fillna(0)

drug,id,atenolol,atorvastatin,bisoprolol,canagliflozin,carvedilol,dapagliflozin,irbesartan,labetalol,losartan,...,metoprolol,nebivolol,olmesartan,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan
index,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0
3,3,0.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,...,0.0,0.0,0.0,0.0,720.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,294,270.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296,296,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,297,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,...,270.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,267.0


In [87]:
df_reshape.shape

(272, 22)

In [88]:
meds1.index

RangeIndex(start=0, stop=272, step=1)

final_meds = pd.merge(df_reshape, meds1, on = 'id')
final_meds.shape

In [89]:
final_meds.head()

,id,atenolol,atorvastatin,bisoprolol,canagliflozin,carvedilol,dapagliflozin,irbesartan,labetalol,losartan,...,olmesartan,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,total_no_of_days_with_drug,total_no_of_days_without_drug
0,0,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,450.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,328
1,1,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,221
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,180,487
3,3,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,301
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,...,NaN,NaN,720.0,NaN,NaN,NaN,NaN,NaN,1620,481


### Download the final meds dataset for reference

In [90]:
final_meds.to_csv("final_meds.csv", index=False)

### 4. All Lab Blood Test of CKD patients summary

In [91]:
structured_creatinine = prep_lab_test_df(creatinine, "creatinine")
structured_glucose = prep_lab_test_df(glucose, "glucose")
structured_HGB = prep_lab_test_df(HGB, "Hemoglobin")
structured_ldl = prep_lab_test_df(ldl, "low-density lipoprotein")
structured_SBP = prep_lab_test_df(SBP, "systolic blood pressure")
structured_DBP = prep_lab_test_df(DBP, "Diastolic Blood Pressure")

Summary of creatinine lab test below:
               id  no_of_test  mean_value  duration_of_test
count  300.000000  300.000000  300.000000        300.000000
mean   149.500000    4.796667    1.319237       1409.396667
std     86.746758    1.288743    0.308531        557.715514
min      0.000000    2.000000    0.770000          6.000000
25%     74.750000    4.000000    1.108125       1081.500000
50%    149.500000    5.000000    1.271000       1322.000000
75%    224.250000    5.000000    1.504250       1767.250000
max    299.000000   10.000000    2.796000       3765.000000

 
Shape of creatinine lab test below:
(300, 4)

 
First four records of creatinine lab test below:
   id  no_of_test  mean_value  duration_of_test
0   0           6    1.236667              2043
1   1           5    1.838000              1214
2   2           5    2.170000              1737
3   3           5    0.996000              1800
4   4           6    1.150000              2120
Summary of glucose lab test below:

In [92]:
structured_creatinine.rename(columns={'no_of_test': "creatinine_no_of_test", 
                       'mean_value': "creatinine_mean_value", 
                       'duration_of_test': "creatinine_duration_of_test"}, inplace=True)
structured_glucose.rename(columns={'no_of_test': "glucose_no_of_test", 
                       'mean_value': "glucose_mean_value", 
                       'duration_of_test': "glucose_duration_of_test"}, inplace=True)
structured_HGB.rename(columns={'no_of_test': "HGB_no_of_test", 
                       'mean_value': "HGB_mean_value", 
                       'duration_of_test': "HGB_duration_of_test"}, inplace=True)
structured_ldl.rename(columns={'no_of_test': "ldl_no_of_test", 
                       'mean_value': "ldl_mean_value", 
                       'duration_of_test': "ldl_duration_of_test"}, inplace=True)
structured_SBP.rename(columns={'no_of_test': "SBP_no_of_test", 
                       'mean_value': "SBP_mean_value", 
                       'duration_of_test': "SBP_duration_of_test"}, inplace=True)
structured_DBP.rename(columns={'no_of_test': "DBP_no_of_test", 
                       'mean_value': "DBP_mean_value", 
                       'duration_of_test': "DBP_duration_of_test"}, inplace=True)

In [93]:
structured_creatinine.head()

,id,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test
0,0,6,1.236667,2043
1,1,5,1.838000,1214
2,2,5,2.170000,1737
3,3,5,0.996000,1800
4,4,6,1.150000,2120


In [94]:
structured_glucose.head()

,id,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test
0,0,6,6.456667,2043
1,1,5,9.240000,1602
2,2,3,6.466667,804
3,3,6,6.525000,2475
4,4,7,9.720000,1974


In [95]:
structured_HGB.head()

,id,HGB_no_of_test,HGB_mean_value,HGB_duration_of_test
0,0,9,13.262222,5405
1,1,7,13.662857,4777
2,2,8,15.700000,4500
3,3,10,13.198000,7395
4,4,5,12.658000,2990


In [96]:
structured_ldl.head()

,id,ldl_no_of_test,ldl_mean_value,ldl_duration_of_test
0,0,3,143.593333,489
1,1,5,82.424000,1602
2,2,3,75.283333,681
3,3,5,98.246000,1670
4,4,4,65.907500,1016


In [97]:
structured_SBP.head()

,id,SBP_no_of_test,SBP_mean_value,SBP_duration_of_test
0,0,8,139.610000,3169
1,1,7,150.232857,2454
2,2,4,140.210000,1242
3,3,7,139.588571,2963
4,4,7,160.667143,2445


In [98]:
structured_DBP.head()

,id,DBP_no_of_test,DBP_mean_value,DBP_duration_of_test
0,0,8,83.173750,3169
1,1,7,80.455714,2454
2,2,4,89.217500,1242
3,3,7,74.857143,2963
4,4,7,97.385714,2445


### Join all the Lab blood test dataset

In [99]:
df1 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(structured_creatinine, structured_glucose, on = 'id'), structured_HGB, on = 'id'), structured_ldl, on = 'id'),  structured_SBP, on = 'id'), structured_DBP, on = 'id')
df2 = pd.merge(demo, df1, on = 'id')

In [100]:
df2.shape

(300, 22)

In [101]:
df2.head()

,id,race,gender,age,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test,...,HGB_duration_of_test,ldl_no_of_test,ldl_mean_value,ldl_duration_of_test,SBP_no_of_test,SBP_mean_value,SBP_duration_of_test,DBP_no_of_test,DBP_mean_value,DBP_duration_of_test
0,0,Unknown,Male,70,6,1.236667,2043,6,6.456667,2043,...,5405,3,143.593333,489,8,139.610000,3169,8,83.173750,3169
1,1,White,Female,71,5,1.838000,1214,5,9.240000,1602,...,4777,5,82.424000,1602,7,150.232857,2454,7,80.455714,2454
2,2,White,Female,57,5,2.170000,1737,3,6.466667,804,...,4500,3,75.283333,681,4,140.210000,1242,4,89.217500,1242
3,3,White,Male,74,5,0.996000,1800,6,6.525000,2475,...,7395,5,98.246000,1670,7,139.588571,2963,7,74.857143,2963
4,4,White,Female,50,6,1.150000,2120,7,9.720000,1974,...,2990,4,65.907500,1016,7,160.667143,2445,7,97.385714,2445


### Now Create a consolidated CKD Dataset for training, testing and building model

In [102]:
 consolidate_ckd = pd.merge(pd.merge(df2, final_meds, on = 'id', how='left'), stage, on = 'id')

In [103]:
consolidate_ckd.fillna(0)
consolidate_ckd.shape

(300, 46)

In [104]:
consolidate_ckd.head()

,id,race,gender,age,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test,...,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,total_no_of_days_with_drug,total_no_of_days_without_drug,Stage_Progress
0,0,Unknown,Male,70,6,1.236667,2043,6,6.456667,2043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290.0,328.0,True
1,1,White,Female,71,5,1.838000,1214,5,9.240000,1602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,221.0,False
2,2,White,Female,57,5,2.170000,1737,3,6.466667,804,...,NaN,NaN,NaN,180.0,NaN,NaN,NaN,180.0,487.0,True
3,3,White,Male,74,5,0.996000,1800,6,6.525000,2475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,301.0,False
4,4,White,Female,50,6,1.150000,2120,7,9.720000,1974,...,NaN,720.0,NaN,NaN,NaN,NaN,NaN,1620.0,481.0,True


### Save the Consolidated CKD Dataset in csv format for further processing

In [105]:
consolidate_ckd.to_csv("consolidate_ckd.csv", index = False)